In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 그래프 기본 테마 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats
from sklearn.preprocessing import LabelEncoder


### 4. 청구입금정보

In [2]:
# 데이터 불러오기
df407 = pd.read_parquet('open/train/4.청구입금정보/201807_train_청구정보.parquet')
df408 = pd.read_parquet('open/train/4.청구입금정보/201808_train_청구정보.parquet')
df409 = pd.read_parquet('open/train/4.청구입금정보/201809_train_청구정보.parquet')
df410 = pd.read_parquet('open/train/4.청구입금정보/201810_train_청구정보.parquet')
df411 = pd.read_parquet('open/train/4.청구입금정보/201811_train_청구정보.parquet')
df412 = pd.read_parquet('open/train/4.청구입금정보/201812_train_청구정보.parquet')

df407

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,5,3,0,0,1,0,3,0,0
1,201807,TRAIN_000001,13,자동이체,주거지,우편,우편,1,1,1,...,163,6,3,0,0,0,0,0,0,0
2,201807,TRAIN_000002,1,자동이체,미확인,이메일,이메일,1,1,1,...,0,6,3,0,0,0,0,121,0,50
3,201807,TRAIN_000003,5,자동이체,주거지,우편,우편,1,1,1,...,0,5,3,2,0,0,0,3,0,2
4,201807,TRAIN_000004,13,자동이체,주거지,우편,우편,0,1,1,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,201807,TRAIN_399995,25,자동이체,주거지,우편,우편,0,0,0,...,0,0,0,0,0,0,0,0,0,0
399996,201807,TRAIN_399996,20,자동이체,미확인,이메일,이메일,1,1,1,...,256,6,3,0,0,0,0,403,0,122
399997,201807,TRAIN_399997,20,자동이체,회사,우편,우편,1,1,1,...,0,5,3,0,0,1,0,0,0,0
399998,201807,TRAIN_399998,20,자동이체,주거지,우편,우편,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# 4번 데이터 병합
df4 = pd.concat([df407, df408, df409, df410, df411], ignore_index=True)
df4 = df4.sort_values(by=['ID', '기준년월']).reset_index(drop=True)
df4

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,5,3,0,0,1,0,3,0,0
1,201808,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,5,3,0,0,1,0,3,0,0
2,201809,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,6,3,0,0,0,0,0,0,0
3,201810,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,6,3,0,0,0,0,0,0,0
4,201811,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,0,6,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,201807,TRAIN_399999,23,자동이체,주거지,우편,우편,1,1,1,...,0,6,3,0,0,0,0,0,0,0
1999996,201808,TRAIN_399999,23,자동이체,주거지,우편,우편,1,1,1,...,0,6,3,0,0,0,0,0,0,0
1999997,201809,TRAIN_399999,23,자동이체,주거지,우편,우편,1,1,1,...,0,6,3,0,0,0,0,0,0,0
1999998,201810,TRAIN_399999,23,자동이체,주거지,우편,우편,1,1,1,...,0,6,3,0,0,0,0,0,0,0


In [4]:
# segment 데이터 불러오기
segment = pd.read_parquet('open/train/1.회원정보/201807_train_회원정보.parquet')
segment_col = segment[['ID', 'Segment']]

In [5]:
# segment left outer join
df4 = df4.merge(segment_col, on='ID', how='left')
df4.head()

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액,Segment
0,201807,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,5,3,0,0,1,0,3,0,0,D
1,201808,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,5,3,0,0,1,0,3,0,0,D
2,201809,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,6,3,0,0,0,0,0,0,0,D
3,201810,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,6,3,0,0,0,0,0,0,0,D
4,201811,TRAIN_000000,27,자동이체,미확인,당사페이앱+이메일,문자메세지,1,1,1,...,6,3,0,0,0,0,0,0,0,D


In [6]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 47 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   기준년월              int64 
 1   ID                object
 2   대표결제일             int64 
 3   대표결제방법코드          object
 4   대표청구지고객주소구분코드     object
 5   대표청구서수령지구분코드      object
 6   청구서수령방법           object
 7   청구서발송여부_B0        int64 
 8   청구서발송여부_R3M       int64 
 9   청구서발송여부_R6M       int64 
 10  청구금액_B0           int64 
 11  청구금액_R3M          int64 
 12  청구금액_R6M          int64 
 13  포인트_마일리지_건별_B0M   int64 
 14  포인트_마일리지_건별_R3M   int64 
 15  포인트_포인트_건별_B0M    int64 
 16  포인트_포인트_건별_R3M    int64 
 17  포인트_마일리지_월적립_B0M  int64 
 18  포인트_마일리지_월적립_R3M  int64 
 19  포인트_포인트_월적립_B0M   int64 
 20  포인트_포인트_월적립_R3M   int64 
 21  포인트_적립포인트_R12M    int64 
 22  포인트_적립포인트_R3M     int64 
 23  포인트_이용포인트_R12M    int64 
 24  포인트_이용포인트_R3M     int64 
 25  포인트_잔여포인트_B0M     int64 
 26  마일_적립포인트_R12M     int64 
 27  마일_적립포인트_R3M

In [7]:
# 결측치 확인: 없음
print(df4.isnull().sum())

기준년월                0
ID                  0
대표결제일               0
대표결제방법코드            0
대표청구지고객주소구분코드       0
대표청구서수령지구분코드        0
청구서수령방법             0
청구서발송여부_B0          0
청구서발송여부_R3M         0
청구서발송여부_R6M         0
청구금액_B0             0
청구금액_R3M            0
청구금액_R6M            0
포인트_마일리지_건별_B0M     0
포인트_마일리지_건별_R3M     0
포인트_포인트_건별_B0M      0
포인트_포인트_건별_R3M      0
포인트_마일리지_월적립_B0M    0
포인트_마일리지_월적립_R3M    0
포인트_포인트_월적립_B0M     0
포인트_포인트_월적립_R3M     0
포인트_적립포인트_R12M      0
포인트_적립포인트_R3M       0
포인트_이용포인트_R12M      0
포인트_이용포인트_R3M       0
포인트_잔여포인트_B0M       0
마일_적립포인트_R12M       0
마일_적립포인트_R3M        0
마일_이용포인트_R12M       0
마일_이용포인트_R3M        0
마일_잔여포인트_B0M        0
할인건수_R3M            0
할인금액_R3M            0
할인건수_B0M            0
할인금액_B0M            0
할인금액_청구서_R3M        0
할인금액_청구서_B0M        0
상환개월수_결제일_R6M       0
상환개월수_결제일_R3M       0
선결제건수_R6M           0
선결제건수_R3M           0
연체건수_R6M            0
연체건수_R3M            0
혜택수혜금액_R3M          0
포인트_마일리지_환산_B0M     0
혜택수혜금액    

In [8]:
df = df4.copy()

# Segment 라벨 인코딩
le_segment = LabelEncoder()
df['Segment_enc'] = le_segment.fit_transform(df['Segment'])

# 할인건수_R3M, 할인건수_B0M 숫자 변환 시도 (에러나는 값은 NaN 처리)
df['할인건수_R3M'] = pd.to_numeric(df['할인건수_R3M'], errors='coerce')
df['할인건수_B0M'] = pd.to_numeric(df['할인건수_B0M'], errors='coerce')

# 숫자형 컬럼만 선택
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Segment_enc가 없으면 추가
if 'Segment_enc' not in numeric_cols:
    numeric_cols.append('Segment_enc')

# 숫자형 컬럼으로 상관계수 계산
corr = df[numeric_cols].corr()

# Segment_enc와 다른 변수들 상관계수 출력
segment_corr = corr['Segment_enc'].sort_values(ascending=False)
print(segment_corr)

# 소수점 4자리 고정 출력 설정
pd.set_option('display.float_format', '{:.2f}'.format)

print(segment_corr)

Segment_enc         1.000000e+00
대표결제일               3.624233e-03
포인트_잔여포인트_B0M       2.816095e-03
기준년월                9.702586e-11
연체건수_R3M           -5.524745e-02
연체건수_R6M           -6.085389e-02
포인트_포인트_월적립_B0M    -1.165825e-01
포인트_포인트_월적립_R3M    -1.257010e-01
상환개월수_결제일_R3M      -1.342887e-01
포인트_포인트_건별_B0M     -1.441895e-01
포인트_이용포인트_R12M     -1.447179e-01
청구서발송여부_R6M        -1.466966e-01
포인트_포인트_건별_R3M     -1.467387e-01
선결제건수_R3M          -1.468056e-01
상환개월수_결제일_R6M      -1.521826e-01
선결제건수_R6M          -1.590074e-01
포인트_이용포인트_R3M      -1.707704e-01
포인트_적립포인트_R3M      -1.723180e-01
포인트_마일리지_월적립_B0M   -1.756440e-01
포인트_적립포인트_R12M     -1.765641e-01
할인금액_B0M           -1.795955e-01
포인트_마일리지_월적립_R3M   -1.813705e-01
청구서발송여부_R3M        -1.825788e-01
마일_잔여포인트_B0M       -1.841356e-01
할인금액_R3M           -1.858640e-01
할인금액_청구서_B0M       -1.957122e-01
할인금액_청구서_R3M       -2.027136e-01
청구서발송여부_B0         -2.088005e-01
마일_이용포인트_R12M      -2.120419e-01
마일_이용포인트_R3M       -2.132017e-01
마일_적립포인트_R

In [9]:
# 상관계수 결과를 딕셔너리 형태로 작성 (예시)
corr_dict = {
    '기준년월': 0.00,
    'ID': np.nan,
    '대표결제일': 0.00,
    '대표결제방법코드': np.nan,
    '대표청구지고객주소구분코드': np.nan,
    '대표청구서수령지구분코드': np.nan,
    '청구서수령방법': np.nan,
    '청구서발송여부_B0': -0.21,
    '청구서발송여부_R3M': -0.18,
    '청구서발송여부_R6M': -0.15,
    '청구금액_B0': -0.58,
    '청구금액_R3M': -0.59,
    '청구금액_R6M': -0.60,
    '포인트_마일리지_건별_B0M': -0.22,
    '포인트_마일리지_건별_R3M': -0.22,
    '포인트_포인트_건별_B0M': -0.14,
    '포인트_포인트_건별_R3M': -0.15,
    '포인트_마일리지_월적립_B0M': -0.18,
    '포인트_마일리지_월적립_R3M': -0.18,
    '포인트_포인트_월적립_B0M': -0.12,
    '포인트_포인트_월적립_R3M': -0.13,
    '포인트_적립포인트_R12M': -0.18,
    '포인트_적립포인트_R3M': -0.17,
    '포인트_이용포인트_R12M': -0.14,
    '포인트_이용포인트_R3M': -0.17,
    '포인트_잔여포인트_B0M': 0.00,
    '마일_적립포인트_R12M': -0.21,
    '마일_적립포인트_R3M': -0.28,
    '마일_이용포인트_R12M': -0.21,
    '마일_이용포인트_R3M': -0.21,
    '마일_잔여포인트_B0M': -0.18,
    '할인건수_R3M': np.nan,
    '할인금액_R3M': -0.19,
    '할인건수_B0M': np.nan,
    '할인금액_B0M': -0.18,
    '할인금액_청구서_R3M': -0.20,
    '할인금액_청구서_B0M': -0.20,
    '상환개월수_결제일_R6M': -0.15,
    '상환개월수_결제일_R3M': -0.13,
    '선결제건수_R6M': -0.16,
    '선결제건수_R3M': -0.15,
    '연체건수_R6M': -0.06,
    '연체건수_R3M': -0.06,
    '혜택수혜금액_R3M': -0.36,
    '포인트_마일리지_환산_B0M': -0.28,
    '혜택수혜금액': -0.35,
}

# DataFrame 생성
corr_df = pd.DataFrame(list(corr_dict.items()), columns=['변수명', 'Segment_enc_상관계수'])

# 엑셀 파일로 저장
corr_df.to_excel('open.xlsx', index=False)
